In [6]:
from sklearn.preprocessing import MinMaxScaler
import itertools
import pandas as pd
import torch

# get our predefined functions (for data processing, etc.)
import utils as ut

# get our predefined quantum circuits and neural networks
import CustomNetworks as cn
import QuantumNN as qnn


EXPERIMENT_ID = 1

X_PATH = "data/X_apd_class6_7.npy"
Y_PATH = "data/y_apd_class6_7.npy"
BIN_ENCODING = {6: 0, 7: 1}


N_QUBITS = 4
FEAT_PER_QUBIT = 4

SCALER = MinMaxScaler()

SUBSET = None

EPOCHS = 16
MODEL_PATH = f"results/models/"
RESULTS_PATH = f"results/model_results_expID_{EXPERIMENT_ID}.csv"
RESULTS_DF = pd.DataFrame()

VALIDATION = True

# for printing colored text output during training
RED_TEXT = "\033[91m"
GREEN_TEXT = "\033[92m"
RESET_COLOR = "\033[0m" 


In [7]:
# load the data
X_train, X_val, X_test, y_train, y_val, y_test, _ = ut.load_and_prepare_data(X_PATH, Y_PATH, scaler=SCALER, bin_encoding=BIN_ENCODING, subset=SUBSET)

# check shapes
print("train:\t", X_train.shape, y_train.shape)
print("val:\t", X_val.shape, y_val.shape)
print("test:\t", X_test.shape, y_test.shape)

train:	 torch.Size([9746, 16]) torch.Size([9746])
val:	 torch.Size([1219, 16]) torch.Size([1219])
test:	 torch.Size([1218, 16]) torch.Size([1218])


In [8]:
cn.get_custom_circuits()

[CustomNetworks.AmplitudeEncodingCircuit,
 CustomNetworks.Anusha2024Circuit,
 CustomNetworks.DoubleEntanglementVLCircuit,
 CustomNetworks.Ranga2024Circuit,
 CustomNetworks.Senokosov2024Circuit]

In [9]:
circuits = [
    cn.Ranga2024Circuit,
    cn.Senokosov2024Circuit,
    cn.Anusha2024Circuit,
    cn.DoubleEntanglementVLCircuit,
]

runs = list(range(1, 11)) # runs are also used as seeds (reproducibility)
learning_rates = [0.01, 0.001]

In [10]:
### RUN EXPERIMENT

id = 0
for seed, circuit, lr in itertools.product(runs, circuits, learning_rates):
    print(f"""{RED_TEXT}RUN {id+1}:{RESET_COLOR}\n {circuit.__name__} \n lr: {lr}""")

    # init the model
    model = qnn.QuantumNeuralNetwork(n_qubits=N_QUBITS, features_per_qubit=FEAT_PER_QUBIT, circuit=circuit, seed=seed)

    # train the model (predifined optimizer, etc.)
    trained_model, loss_history, val_loss_history, val_f1s = ut.train_model(
        model, X_train, y_train, X_val, y_val, epochs=EPOCHS, batch_size=32, lr=lr
    )
    
    # save trained model
    model_path = MODEL_PATH + f"model_expID_{EXPERIMENT_ID}_dfID_{id}.pth"
    torch.save(trained_model, model_path)

    # evaluate
    bce, accuracy, precision, recall, f1 = trained_model.evaluate(X_test, y_test)

    results_dict = {
        "id": id,
        "f1_score": f1, "precision": precision, "recall": recall, "accuracy": accuracy, "binary_cross_entropy": bce,
        "seed": seed,
        "circuit": circuit.__name__,
        "learning_rate": lr,
        "loss_history": loss_history,
        "val_loss_history": val_loss_history,
        "val_f1s": val_f1s,
        "scaler" : str(SCALER).replace("()", "")
    }
    print(f"\n{GREEN_TEXT}RESULTS:\n F1-Score:{f1}, Precision:{precision}, Recall:{recall}, Accuracy:{accuracy}, BCE:{bce} {RESET_COLOR} \n\n")
    
    # append the results_dict to the RESULTS_DF
    RESULTS_DF = pd.concat([RESULTS_DF, pd.DataFrame([results_dict])], ignore_index=True)
    RESULTS_DF.to_csv(path_or_buf=RESULTS_PATH) # save for each run (in case of a crash)

    id += 1

RUN 1:
 Ranga2024Circuit 
 lr: 0.01


Training progress: 100%|██████████| 304/304 [01:04<00:00,  4.72batch/s]


Epoch 1/16, Train-Loss: 0.6914, Val-Loss: 0.6703, Val-F1: 0.7289, Val-Precision: 0.6473, Val-Recall: 0.8341, Val-Accuracy: 0.6809


Training progress: 100%|██████████| 304/304 [01:04<00:00,  4.74batch/s]


Epoch 2/16, Train-Loss: 0.6603, Val-Loss: 0.6504, Val-F1: 0.7305, Val-Precision: 0.6577, Val-Recall: 0.8214, Val-Accuracy: 0.6883


...

Epoch 14/16, Train-Loss: 0.4712, Val-Loss: 0.4682, Val-F1: 0.8456, Val-Precision: 0.8081, Val-Recall: 0.8868, Val-Accuracy: 0.8335


Training progress: 100%|██████████| 304/304 [00:24<00:00, 12.43batch/s]


Epoch 15/16, Train-Loss: 0.4650, Val-Loss: 0.4624, Val-F1: 0.8456, Val-Precision: 0.8081, Val-Recall: 0.8868, Val-Accuracy: 0.8335


Training progress: 100%|██████████| 304/304 [00:24<00:00, 12.47batch/s]


Epoch 16/16, Train-Loss: 0.4592, Val-Loss: 0.4570, Val-F1: 0.8461, Val-Precision: 0.8064, Val-Recall: 0.8900, Val-Accuracy: 0.8335

RESULTS:
 F1-Score:0.8493150684931506, Precision:0.811046511627907, Recall:0.8913738019169329, Accuracy:0.8374384236453202, BCE:0.4610368311405182  




In [11]:
RESULTS_DF

,id,f1_score,precision,recall,accuracy,binary_cross_entropy,seed,circuit,learning_rate,loss_history,val_loss_history,val_f1s,scaler
0,0,0.719338,0.679943,0.763578,0.693760,0.630533,1,Ranga2024Circuit,0.010,"[0.6914023258968404, 0.660341840433447, 0.6464...","[0.6702669262886047, 0.6504126787185669, 0.640...","[0.7289198606271777, 0.7304964539007093, 0.725...",MinMaxScaler
1,1,0.743894,0.660471,0.851438,0.698686,0.656471,1,Ranga2024Circuit,0.001,"[0.7414130529290751, 0.7162491987018209, 0.703...","[0.7262354493141174, 0.7087317705154419, 0.699...","[0.6793066088840737, 0.6793066088840737, 0.679...",MinMaxScaler
2,2,0.767611,0.778325,0.757188,0.764368,0.537861,1,Senokosov2024Circuit,0.010,"[0.6834787334266462, 0.6524414312290517, 0.630...","[0.6672486066818237, 0.6451585292816162, 0.626...","[0.5501551189245087, 0.6963350785340314, 0.748...",MinMaxScaler
3,3,0.711150,0.789474,0.646965,0.729885,0.649993,1,Senokosov2024Circuit,0.001,"[0.747941004014329, 0.7088381959811637, 0.6907...","[0.7237253189086914, 0.6978234052658081, 0.686...","[0.6793066088840737, 0.6793066088840737, 0.679...",MinMaxScaler
4,4,0.686275,0.702341,0.670927,0.684729,0.643866,1,Anusha2024Circuit,0.010,"[0.6841570809483528, 0.6475279982152738, 0.639...","[0.661899745464325, 0.6508845090866089, 0.6496...","[0.6795180722891566, 0.6467165419783873, 0.656...",MinMaxScaler
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,75,0.876352,0.848802,0.905751,0.868637,0.535430,10,Senokosov2024Circuit,0.001,"[0.7427862837518516, 0.7225275243583479, 0.704...","[0.7320263385772705, 0.7133990526199341, 0.696...","[0.4785992217898833, 0.22032583397982933, 0.00...",MinMaxScaler
76,76,0.607735,0.600624,0.615016,0.591954,0.671852,10,Anusha2024Circuit,0.010,"[0.6998752198721233, 0.6842588333314971, 0.676...","[0.6892290115356445, 0.6780279874801636, 0.673...","[0.7211231652839821, 0.5921158487530169, 0.675...",MinMaxScaler
77,77,0.616460,0.599698,0.634185,0.594417,0.682844,10,Anusha2024Circuit,0.001,"[0.7172094675663271, 0.7065669209941438, 0.703...","[0.7106428742408752, 0.7054082751274109, 0.702...","[0.6600467289719626, 0.5771012574454004, 0.544...",MinMaxScaler
78,78,0.849772,0.809249,0.894569,0.837438,0.407024,10,DoubleEntanglementVLCircuit,0.010,"[0.5572234008853373, 0.46739204296548115, 0.42...","[0.49795830249786377, 0.44423338770866394, 0.4...","[0.8436308161708619, 0.8469696969696969, 0.854...",MinMaxScaler


In [12]:
# circuit = qc.QuantumCircuit

# # initialize the model
# model = qnn.QuantumNeuralNetwork(n_qubits=N_QUBITS, features_per_qubit=FEAT_PER_QUBIT, circuit=circuit, seed=1)

# # train the model (predifined optimizer, etc.)
# trained_model, loss_history, val_loss_history = ut.train_model(
#     model, X_train, y_train, epochs=EPOCHS, batch_size=64, lr=0.01
# ) #X_val, y_val,


# # Evaluate the model
# bce, accuracy, precision, recall, f1 = trained_model.evaluate(X_test, y_test)
# print(f"Binary Cross-Entropy on test data: {bce:.4f}")
# print(f"Accuracy: {accuracy:.4f}")
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1 Score: {f1:.4f}")

In [13]:
# # To load later
# loaded_model = torch.load("simple_model.pth")